In [129]:
import re
import pdfplumber
from pprint import pprint
import json

In [130]:
file = "TimUnofficialTranscript.pdf"

In [131]:
text = ""
with pdfplumber.open(file) as pdf:
    for page in pdf.pages:
        text += page.extract_text()

In [132]:
honorsRE = re.compile(r'.*honors', re.IGNORECASE)
courseRE = re.compile(r'[A-Z]+\s\d{2,3}')

In [133]:
courseList = []
for line in text.split('\n'):
    if honorsRE.match(line) or courseRE.match(line):
        courseList.append(line)
# pprint(courseList)

In [134]:
newList = []
for entry in courseList:
    if entry == 'Campus HONORS':
        if newList:
            newList[-1] += ' ' + entry
    else:
        newList.append(entry)
# pprint(newList)

In [135]:
honorsList = []
for course in newList:
    if honorsRE.match(course):
        honorsList.append(course)
cleaned = [s.replace('\n', '') for s in honorsList]
# print(cleaned)
pprint(honorsList)

['CSC 150 Kutztown UG ETHICAL LEGAL PROF IMPACT DIG - A 3.000 12.00 Campus '
 'HONORS',
 'CMP 200 Kutztown UG RESEARCH AND COMPOSITION - A 3.000 12.00 Campus HONORS',
 'COM 237 Kutztown UG WOMEN WRITERS IN PERFORMN - A 3.000 12.00 Campus HONORS',
 'HIS 14 Kutztown UG HISTORY OF CIVILIZATION A - A- 3.000 11.01 Campus HONORS',
 'CSC 354 Kutztown UG SOFTWARE ENGINEERING I - A 3.000 12.00 Campus HONORS',
 'CPSC 398 Kutztown Campus UG Honors Supplementary Research 3.000']


In [136]:
courseJSON = []
courseLineRE = re.compile(r'([A-Z]+ \d+) (\w+\s*\w*) ([UGA]*) (\D+?) ([ABCDF-]*)\s?(\d\.\d{3})')
for course in cleaned:
    if courseLineRE.match(course):
        course = courseLineRE.search(course)
        if course:
            courseCode = course.group(1)
            title = course.group(4)
            grade = course.group(5)
            creditHours = course.group(6)
            courseDict = {courseCode: {"title": title, "grade": grade, "credits": creditHours}}
            courseJSON.append(courseDict)

In [137]:
nameRE = re.compile(r'^([A-Za-z]+), ([A-Za-z]+)')
for line in text.split('\n'):
    if nameRE.match(line):
        line = nameRE.search(line)
        if line:
            lastName = line.group(1)
            firstName = line.group(2)
            studentJSON = {firstName + ' ' + lastName: courseJSON}
# pprint(studentJSON)
y = json.dumps(studentJSON)
pprint(studentJSON)

{'Timothy Pasquel': [{'CSC 150': {'credits': '3.000',
                                  'grade': 'A',
                                  'title': 'ETHICAL LEGAL PROF IMPACT DIG -'}},
                     {'CMP 200': {'credits': '3.000',
                                  'grade': 'A',
                                  'title': 'RESEARCH AND COMPOSITION -'}},
                     {'COM 237': {'credits': '3.000',
                                  'grade': 'A',
                                  'title': 'WOMEN WRITERS IN PERFORMN -'}},
                     {'HIS 14': {'credits': '3.000',
                                 'grade': 'A-',
                                 'title': 'HISTORY OF CIVILIZATION A -'}},
                     {'CSC 354': {'credits': '3.000',
                                  'grade': 'A',
                                  'title': 'SOFTWARE ENGINEERING I -'}},
                     {'CPSC 398': {'credits': '3.000',
                                   'grade': '',
         